# Accessing Databases via Web APIs: Lecture Code
* * * * *

Aside from Rochell's tutorial, I added some materials from [Python for  everybody](http://oskicat.berkeley.edu/search~S1?/Xweb+scraping+python&searchscope=1&SORT=D/Xweb+scraping+python&searchscope=1&SORT=D&SUBKEY=web+scraping+python/1%2C9%2C9%2CB/frameset&FF=Xweb+scraping+python&searchscope=1&SORT=D&9%2C9%2C) by Severance and [Web scraping with Python](http://oskicat.berkeley.edu/search~S1?/Xweb+scraping+python&searchscope=1&SORT=D/Xweb+scraping+python&searchscope=1&SORT=D&SUBKEY=web+scraping+python/1%2C9%2C9%2CB/frameset&FF=Xweb+scraping+python&searchscope=1&SORT=D&1%2C1%2C) by Mitchell. 


In [1]:
# Import libraries
import requests
import urllib 
import json
import math
import ssl
import time

## 1. Constructing API GET Request

API (Application Program Interfaces) is basically a "contract" between one application to another. When we use an API, generally one programs (in this case, "New York Times") makes a set of services available for use by other applications and publishes the APIs (i.e., the "rules") that must be followed to access the services provided by the program.

You need an API key to make use of a vendor's API. The vendor wants to know who is using their services and **how much** each user is using. Usually, they have a policy to make you pay for your use or limit the number of requests you can make during a particular time period (i.e., a day).

The workflow is as follows. Every call will require us to provide a) a base URL for the API, b) some authorization code or key, and c) a format for the response.

* Web scraping is a more general data collection technique than using API. Web scraping is when we write a program that pretends to be a web browser and retrieves pages, then examines the data in those pages looking for patterns. Using API is in most cases easier, faster, and legally more safe than web scraping.

* There's a New York Time library called nytimes. But don't use that library since we aim to learn general skills that can be applied to any API step-by-step. Also, you should not use nytimes or any other similar libraries for doing the API group assignment. 


In [2]:
# set key
key="lJWFqmFXKLE8yoxN95xLjnyqMFk67a5I" # Please use your API key.

# set base url
base_url="http://api.nytimes.com/svc/search/v2/articlesearch"

# set response format
response_format=".json"

You often want to send some sort of data in the URL’s query string. This data tells the API what information you want. In our case, we want articles about Prince. Requests allows you to provide these arguments as a dictionary, using the `params` keyword argument. In addition to the search term `q`, we have to put in the `api-key` term.

![How dictionary Works](https://developers.google.com/edu/python/images/dict.png)

In [3]:
# set search parameters
search_params = {"q":"Prince",
                 "api-key":key}       

Now we're ready to make the request. We use the `.get` method from the `requests` library to make an HTTP GET Request.

In [4]:
# make request
nyt_response = requests.get(base_url + response_format, params = search_params)

Now, we have a [response](http://docs.python-requests.org/en/latest/api/#requests.Response) object called `r`. We can get all the information we need from this object. For instance, we can see that the URL has been correctly encoded by printing the URL. Click on the link to see what happens.

In [5]:
print(nyt_response.url)

http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Prince&api-key=lJWFqmFXKLE8yoxN95xLjnyqMFk67a5I


Click on that link to see what it returns!

Hmm - looks like we're getting a lot of junk unrelated to the musical artist in here. Let's make it a little more specific:

In [6]:
# set search parameters
search_params = {"q":"Prince music",
                 "api-key":key}  
# make request
nyt_response = requests.get(base_url + response_format, params=search_params)

print(nyt_response.url)

http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Prince+music&api-key=lJWFqmFXKLE8yoxN95xLjnyqMFk67a5I


Better!

### Challenge 1:  Adding a date range

What if we only want to search within a particular date range? The NYT Article API allows us to specify start and end dates.

Alter the `search_params` code above so that the request only searches for articles in the year 2019.

You're gonna need to look at the documentation [here](http://developer.nytimes.com/docs/read/article_search_api_v2) to see how to do this.

In [7]:
#YOUR CODE HERE

# Uncomment to test
# nyt_response = requests.get(base_url+response_format, params=search_params)
# nyt_response.url

### Challenge 2:  Specifying a results page

The above will return the first 10 results. To get the next ten, you need to add a "page" parameter. Change the search parameters above to get the second 10 resuls. 

In [8]:
search_params = {"q":"Prince music",
                 "begin_date": "20190101",
                 "end_date": "20191231",
                 "page": "1",
                 "api-key":key}  
# make request
nyt_response = requests.get(base_url+response_format, params=search_params)

print(nyt_response.url)

# Uncomment to test
# r = requests.get(base_url+response_format, params=search_params)
# r.url

http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Prince+music&begin_date=20190101&end_date=20191231&page=1&api-key=lJWFqmFXKLE8yoxN95xLjnyqMFk67a5I


## 2. Parsing the response text

Simply put, parsing means making something complex understandable by analyzing its parts (see this [Quora entry](https://www.quora.com/What-exactly-does-parsing-mean-in-programming)).

We can read the content of the server’s response using `.text`

In [9]:
# Inspect the content of the response, parsing the result as text
response_text= nyt_response.text

# Remember string is a sequence of characters.
# So, the result will display 1,000 characters.
print(response_text[:1000])

{"status":"OK","copyright":"Copyright (c) 2019 The New York Times Company. All Rights Reserved.","response":{"docs":[{"web_url":"https://www.nytimes.com/2019/02/10/arts/music/grammy-awards.html","snippet":"At a ceremony dominated by female performers and presenters, Musgraves won album of the year for “Golden Hour,” as well as three other awards.","lead_paragraph":"At a ceremony dominated by female performers and presenters, Musgraves won album of the year for “Golden Hour,” as well as three other awards.","print_page":"1","blog":{},"source":"The New York Times","multimedia":[{"rank":0,"subtype":"xlarge","caption":null,"credit":null,"type":"image","url":"images/2019/02/11/arts/11grammys-live-drake/merlin_150509697_1c8fbc37-51be-43a3-a4fd-8f02e8308856-articleLarge.jpg","height":404,"width":600,"legacy":{"xlarge":"images/2019/02/11/arts/11grammys-live-drake/merlin_150509697_1c8fbc37-51be-43a3-a4fd-8f02e8308856-articleLarge.jpg","xlargewidth":600,"xlargeheight":404},"subType":"xlarge","cr

What you see here is JSON text, encoded as unicode text. JSON stands for "Javascript object notation." It has a very similar structure to a python dictionary -- both are built on key/value pairs. This makes it easy to convert JSON response to a python dictionary.

In [10]:
# Convert JSON response to a dictionary

data = json.loads(response_text)

# Can you understand the data?
data

{'status': 'OK',
 'copyright': 'Copyright (c) 2019 The New York Times Company. All Rights Reserved.',
 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2019/02/10/arts/music/grammy-awards.html',
    'snippet': 'At a ceremony dominated by female performers and presenters, Musgraves won album of the year for “Golden Hour,” as well as three other awards.',
    'lead_paragraph': 'At a ceremony dominated by female performers and presenters, Musgraves won album of the year for “Golden Hour,” as well as three other awards.',
    'print_page': '1',
    'blog': {},
    'source': 'The New York Times',
    'multimedia': [{'rank': 0,
      'subtype': 'xlarge',
      'caption': None,
      'credit': None,
      'type': 'image',
      'url': 'images/2019/02/11/arts/11grammys-live-drake/merlin_150509697_1c8fbc37-51be-43a3-a4fd-8f02e8308856-articleLarge.jpg',
      'height': 404,
      'width': 600,
      'legacy': {'xlarge': 'images/2019/02/11/arts/11grammys-live-drake/merlin_150509697_1c8f

That looks intimidating! But it's really just a big dictionary. Let's see what keys we got in there.

In [11]:
# Extract keys 

data.keys()

dict_keys(['status', 'copyright', 'response'])

In [12]:
# This is boring

data['status']

'OK'

In [13]:
# So is this

data['copyright']

'Copyright (c) 2019 The New York Times Company. All Rights Reserved.'

In [14]:
# this is what we want!
# data['response']

In [15]:
data['response'].keys()

dict_keys(['docs', 'meta'])

In [16]:
data['response']['meta'].keys()

dict_keys(['hits', 'offset', 'time'])

In [17]:
data['response']['meta']['hits'] # Number of hits

50

In [18]:
# data['response']['docs']
type(data['response']['docs'])

list

That looks what we want! Let's put that in it's own variable.

In [19]:
docs = data['response']['docs']

In [20]:
docs[1]

{'web_url': 'https://www.nytimes.com/2019/01/10/arts/music/rat-boy-internationally-unknown.html',
 'snippet': 'The 22-year-old English musician Jordan Cardy sends up authority and celebrates slackerdom in songs that combine the energy of punk with the storytelling of hip-hop.',
 'lead_paragraph': 'The 22-year-old English musician Jordan Cardy sends up authority and celebrates slackerdom in songs that combine the energy of punk with the storytelling of hip-hop.',
 'print_page': '16',
 'blog': {},
 'source': 'The New York Times',
 'multimedia': [{'rank': 0,
   'subtype': 'xlarge',
   'caption': None,
   'credit': None,
   'type': 'image',
   'url': 'images/2019/01/13/arts/13ratboy2/merlin_148718151_8fa1d02f-224d-4e88-8d9c-71a714512b0c-articleLarge.jpg',
   'height': 480,
   'width': 600,
   'legacy': {'xlarge': 'images/2019/01/13/arts/13ratboy2/merlin_148718151_8fa1d02f-224d-4e88-8d9c-71a714512b0c-articleLarge.jpg',
    'xlargewidth': 600,
    'xlargeheight': 480},
   'subType': 'xlarge'

## 3. Putting everything together to get all the articles.

That's great. But we only have 10 items. The original response said we had 50 hits! Which means we have to make 50/10, or 5 requests to get them all.*

* In general, most free APIs have limits on how often you can "call" them, i.e., how many requests you can send. For the NYT, the daily limit is 1000, and you are limited to 5 calls/sec. At first, this might sound like plenty--but in general, it's not. You'll want to be creative with your search terms and date restrictions to ensure that you have a manageable number of calls to the API. 

* Sounds like a job for a loop! 

But first, let's review what we've done so far.

In [21]:
# set key
key="lJWFqmFXKLE8yoxN95xLjnyqMFk67a5I"

# set base url
base_url="http://api.nytimes.com/svc/search/v2/articlesearch"

# set response format
response_format=".json"

# set search parameters
search_params = {"q":"Prince music",
                 "api-key":key,
                 "begin_date": "20190101",
                 "end_date": "20191231"}

# make request
rr = requests.get(base_url + response_format, params = search_params)
    
# convert to a dictionary
data = json.loads(rr.text)
    
# get number of hits
hits = data['response']['meta']['hits']
print("number of hits: " + str(hits))
    
# get number of pages
# math.ceil(x) Return the ceiling of x as a float
pages = int(math.ceil(hits/10))

number of hits: 50


In [22]:
# make an empty list where we'll hold all of our docs for every page
all_docs = [] 
    
# now we're ready to loop through the pages
for i in range(pages):
    print("collecting page " + str(i))
        
    # set the page parameter
    search_params['page'] = i
        
    # make request
    rr = requests.get(base_url + response_format, params = search_params)
    
    # get text and convert to a dictionary
    data = json.loads(rr.text)
        
    # get just the docs
    docs = data['response']['docs']
        
    # add those docs to the big list
    all_docs = all_docs + docs # or you can do all_docs += docs

collecting page 0
collecting page 1
collecting page 2
collecting page 3
collecting page 4


In [23]:
print(data)

{'status': 'OK', 'copyright': 'Copyright (c) 2019 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2019/02/27/fashion/weddings/taylor-swift-surprises-a-couple-again.html', 'snippet': 'An Oscar weekend to remember: These two and their guests got a live performance of “King of My Heart” at their engagement party.', 'lead_paragraph': 'An Oscar weekend to remember: These two and their guests got a live performance of “King of My Heart” at their engagement party.', 'blog': {}, 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2019/02/27/fashion/weddings/27doozy1/27doozy1-articleLarge-v2.jpg', 'height': 400, 'width': 600, 'legacy': {'xlarge': 'images/2019/02/27/fashion/weddings/27doozy1/27doozy1-articleLarge-v2.jpg', 'xlargewidth': 600, 'xlargeheight': 400}, 'subType': 'xlarge', 'crop_name': 'articleLarge'}, {'rank': 0, 'subtype': 'wide',

In [24]:
len(all_docs)

50

### Challenge 3: Make a function

Turn the code above into a function that inputs a search term, and returns all the documents containing that search term in 2014.

In [25]:
def apisearch(q, begin):
    # set key
    key="c98c46b9278b4f4e98acb812ec3d9dbb"
    # set base url
    base_url="http://api.nytimes.com/svc/search/v2/articlesearch"
    # set response format
    response_format=".json" 
    search_params = {"q": q,
                 "begin_date": begin,
                 "end_date": "20141231",
                 "page": "1",
                 "api-key":key}  
    # make request
    r = requests.get(base_url+response_format, params=search_params)
    print(r.url)

In [26]:
apisearch("Clinton", "20140101")

http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Clinton&begin_date=20140101&end_date=20141231&page=1&api-key=c98c46b9278b4f4e98acb812ec3d9dbb


## 4. Formatting and Exporting

Let's take another look at one of these documents.

In [27]:
all_docs[0]

{'web_url': 'https://www.nytimes.com/aponline/2019/03/01/arts/ap-us-people-manuela-testolini.html',
 'snippet': 'Andra Day, Dave Stewart of the Eurythmics and Philip Bailey and Verdine White of Earth, Wind &amp; Fire will perform during a fundraising gala honoring Prince to benefit the foundation of his second wife, Manuela Testolini.',
 'lead_paragraph': 'Andra Day, Dave Stewart of the Eurythmics and Philip Bailey and Verdine White of Earth, Wind &amp; Fire will perform during a fundraising gala honoring Prince to benefit the foundation of his second wife, Manuela Testolini.',
 'blog': {},
 'source': 'AP',
 'multimedia': [],
 'headline': {'main': "Foundation of Prince's Second Wife to Honor Him at Gala",
  'kicker': None,
  'content_kicker': None,
  'print_headline': "Foundation of Prince's Second Wife to Honor Him at Gala",
  'name': None,
  'seo': None,
  'sub': None},
 'keywords': [],
 'pub_date': '2019-03-01T22:05:08+0000',
 'document_type': 'article',
 'news_desk': 'None',
 'sect

This is all great, but it's pretty messy. What we’d really like to to have, eventually, is a CSV, with each row representing an article, and each column representing something about that article (header, date, etc). As we saw before, the best way to do this is to make a lsit of dictionaries, with each dictionary representing an article and each dictionary representing a field of metadata from that article (e.g. headline, date, etc.) We can do this with a custom function:

In [28]:
def format_articles(unformatted_docs):
    '''
    This function takes in a list of documents returned by the NYT api 
    and parses the documents into a list of dictionaries, 
    with 'id', 'header', and 'date' keys
    '''
    formatted = []
    for i in unformatted_docs:
        dic = {}
        dic['id'] = i['_id']
        dic['headline'] = i['headline']['main'].encode("utf8")
        dic['date'] = i['pub_date'][0:10] # cutting time of day.
        formatted.append(dic)
    return(formatted) 

In [29]:
all_formatted = format_articles(all_docs)

In [30]:
all_formatted[:5]

[{'id': '5c79acee49f0eacbf105af2d',
  'headline': b"Foundation of Prince's Second Wife to Honor Him at Gala",
  'date': '2019-03-01'},
 {'id': '5c7e8fda49f0eacbf105b6d4',
  'headline': b'Queen Marks 50th Anniversary of Investiture of Son Charles as Prince of Wales',
  'date': '2019-03-05'},
 {'id': '5c2e85e53a125f5075c02bdf',
  'headline': b'12 Pop, Rock and Jazz Concerts to Check Out in N.Y.C. This Weekend',
  'date': '2019-01-03'},
 {'id': '5c2fbcb23a125f5075c02e49',
  'headline': b'New Year\xe2\x80\x99s Sounds: The Week in Classical Music',
  'date': '2019-01-04'},
 {'id': '5c6d176f3a125f5075c0a8b6',
  'headline': b'Chaka Khan on Artistic Freedom, Her Side Hustles and Joni Mitchell',
  'date': '2019-02-20'}]

### Challenge 4: Export the data to a CSV.

In [31]:
## Import library csv 
import csv 

## Extract keys from the dictionary 
keys = all_formatted[0].keys() 

## Save the keys as a csv file
with open('article_API.csv', 'w') as output_file:
    # Maps Dictionaries onto Objects
    dict_writer = csv.DictWriter(f = output_file, fieldnames = keys)
    # Write rows with field names
    dict_writer.writeheader()
    # Write rows of data
    dict_writer.writerows(all_formatted) 
    